In [3]:
# -*- coding: utf-8 -*-
import sqlite3
import pandas as pd
import re

# Simplify genre function with NaN handling
def simplify_genre(genre, genre_list):
    if pd.isna(genre):
        return 'unknown'  # or any default genre you prefer
    genre = genre.lower()

    # Simplify the genre based on the list
    for simplified_genre in genre_list:
        if simplified_genre in genre:
            return simplified_genre

    # Return the original genre if no match is found
    return genre

# Load cleaned datasets
spotify_data = pd.read_csv("/content/Spotify.csv") #these are the paths to my files, may be different on yours.
apple_data = pd.read_csv("/content/Apple.csv")
top_songs = pd.read_csv("/content/RecentPlayed.csv")

# Standardize column names
spotify_data.rename(columns={'artists': 'artist', 'track_name': 'song', 'track_genre': 'genre'}, inplace=True)
apple_data.rename(columns={'artistName': 'artist', 'trackName': 'song', 'primaryGenreName': 'genre'}, inplace=True)

# Connect to SQLite database
conn = sqlite3.connect('music.db')

# Save datasets to SQLite database
spotify_data.to_sql('Spotify', conn, if_exists='replace', index=False)
apple_data.to_sql('Apple', conn, if_exists='replace', index=False)
top_songs.to_sql('TopSongs', conn, if_exists='replace', index=False)

# Query to get distinct genres from Spotify
query = "SELECT DISTINCT genre FROM Spotify"
distinct_genres = pd.read_sql_query(query, conn)

# Convert distinct genres to list
distinct_genres_list = distinct_genres['genre'].tolist()

# Apply the genre simplification to top_songs dataset
top_songs['simplified_genre'] = top_songs['genre'].apply(lambda x: simplify_genre(x, distinct_genres_list))

# Select only the top 3 songs
top_5_songs = top_songs.head(3)
top_5_songs.to_sql('TopSongs', conn, if_exists='replace', index=False)

# Function to get the popular artists you may like
def get_popular_artists(limit=10):
    query = """
    SELECT artist, SUM(popularity) AS total_popularity, genre
    FROM Spotify
    WHERE genre IN (SELECT DISTINCT simplified_genre FROM TopSongs)
    GROUP BY artist, genre  -- Group by both artist and genre
    ORDER BY total_popularity DESC
    LIMIT ?;
    """

    try:
        popular_artists = pd.read_sql_query(query, conn, params=[limit])
        return popular_artists

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to fetch recommendations
def get_recommendations(song_name=None, artist=None, genre=None, limit=20):
    query = """
    WITH SpotifyRecommendations AS (
        SELECT
            TS.song AS Original_Song,
            TS.artist AS Original_Artist,
            TS.simplified_genre AS Original_Genre,
            S.song AS Spotify_Song,
            S.artist AS Spotify_Artist,
            S.genre AS Spotify_Genre,
            S.popularity AS Spotify_Popularity,
            ROW_NUMBER() OVER (PARTITION BY TS.song ORDER BY S.popularity DESC) AS rank
        FROM TopSongs TS
        JOIN Spotify S ON
            (LOWER(TS.simplified_genre) = LOWER(S.genre) OR
             LOWER(S.genre) LIKE '%' || LOWER(TS.simplified_genre) || '%')
        WHERE
            LOWER(S.artist) != LOWER(TS.artist)
            AND S.genre IS NOT NULL
        GROUP BY TS.song, S.song, S.artist
    ),
    AppleRecommendations AS (
        SELECT
            TS.song AS Original_Song,
            TS.artist AS Original_Artist,
            TS.simplified_genre AS Original_Genre,
            A.song AS Apple_Song,
            A.artist AS Apple_Artist,
            A.genre AS Apple_Genre,
            ROW_NUMBER() OVER (PARTITION BY TS.song ORDER BY A.song) AS rank
        FROM TopSongs TS
        JOIN Apple A ON
            (LOWER(TS.simplified_genre) = LOWER(A.genre) OR
             LOWER(A.genre) LIKE '%' || LOWER(TS.simplified_genre) || '%')
        WHERE
            LOWER(A.artist) != LOWER(TS.artist)
            AND A.genre IS NOT NULL
        GROUP BY TS.song, A.song, A.artist
    )
    SELECT
        SR.Original_Song,
        SR.Original_Artist,
        SR.Original_Genre,
        SR.Spotify_Song,
        SR.Spotify_Artist,
        SR.Spotify_Genre,
        SR.Spotify_Popularity,
        AR.Apple_Song,
        AR.Apple_Artist,
        AR.Apple_Genre
    FROM SpotifyRecommendations SR
    LEFT JOIN AppleRecommendations AR ON
        SR.Original_Song = AR.Original_Song AND SR.rank = AR.rank
    WHERE
        SR.rank <= 3
    ORDER BY
        SR.Original_Song, SR.rank
    """

    try:
        recommendations = pd.read_sql_query(query, conn)

        if recommendations.empty:
            print("No recommendations found.")
            return None

        return recommendations

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Get the artists
popular_artists = get_popular_artists(limit=10)  # Adjust limit to what you want it to output

if popular_artists is not None:
    print("\nPopular Artists You May Like:\n")
    for idx, artist_row in popular_artists.iterrows():
        print(f"    - {artist_row['artist']} (Total Popularity: {artist_row['total_popularity']}, Genre: {artist_row['genre']})")

# Get recommendations
recommendations = get_recommendations()

if recommendations is not None:
    print("\nRecommendations Based on Your Top 5 Personal Songs:\n")

    grouped = recommendations.groupby(['Original_Song', 'Original_Artist', 'Original_Genre'])

    for (song, artist, genre), group in grouped:
        print(f"\nFor Your Song: {song} by {artist} (Genre: {genre})")
        print("Recommendations:")

        for idx, rec in group.iterrows():
            print(f"\n    Recommendation {idx + 1}:")
            print(f"        Spotify: '{rec['Spotify_Song']}' by {rec['Spotify_Artist']}")
            print(f"        Spotify Genre: {rec['Spotify_Genre']}")
            print(f"        Spotify Popularity: {rec['Spotify_Popularity']}")
            print(f"        Apple Music: '{rec['Apple_Song']}' by {rec['Apple_Artist']}")
            print(f"        Apple Music Genre: {rec['Apple_Genre']}")

    # Optional: Save recommendations to CSV
    recommendations.to_csv("/content/SQLite_Recommendations.csv", index=False)
    print("\nRecommendations saved to SQLite_Recommendations.csv")

# Close the connection
conn.close()


Popular Artists You May Like:

    - Travis Scott (Total Popularity: 1124, Genre: house)
    - The Chainsmokers (Total Popularity: 726, Genre: house)
    - Alan Walker (Total Popularity: 713, Genre: house)
    - Arijit Singh (Total Popularity: 677, Genre: pop)
    - BTS (Total Popularity: 650, Genre: pop)
    - Ozuna (Total Popularity: 630, Genre: latin)
    - Darshan Raval (Total Popularity: 539, Genre: pop)
    - Taylor Swift (Total Popularity: 510, Genre: pop)
    - Sidhu Moose Wala (Total Popularity: 505, Genre: pop)
    - BLACKPINK (Total Popularity: 492, Genre: pop)

Recommendations Based on Your Top 5 Personal Songs:


For Your Song: COMPETENCIA by Myke Towers (Genre: latin)
Recommendations:

    Recommendation 1:
        Spotify: 'La Bachata' by Manuel Turizo
        Spotify Genre: latin
        Spotify Popularity: 98
        Apple Music: 'Almost Like Praying (feat. Artists for Puerto Rico)' by Lin-Manuel Miranda
        Apple Music Genre: Pop Latino

    Recommendation 2:
   